In [5]:
# 使用submitter对api进行访问
from sparksampling import Submitter
from sparksampling.utilities.var import FILE_TYPE_CSV
from sparksampling.utilities.var import SIMPLE_RANDOM_SAMPLING_METHOD
submitter = Submitter()
dataset_uri = 'hdfs://localhost:9000/dataset/titanic/train.csv'
fraction = 0.4

In [2]:
# 提交抽样任务
submit_response = submitter.submit_sampling_simplejob(dataset_uri,
                                              method=SIMPLE_RANDOM_SAMPLING_METHOD,
                                              file_type=FILE_TYPE_CSV,
                                              fraction=fraction,
                                              with_header=True)
job_id = submit_response.job_id
submit_response.to_dict()

request: http://localhost:8000/v1/sampling/simplejob/
2021-03-23 16:26:50,614 DEBUG connectionpool.py[231] Starting new HTTP connection (1): localhost:8000
2021-03-23 16:26:50,695 DEBUG connectionpool.py[461] http://localhost:8000 "POST /v1/sampling/simplejob/ HTTP/1.1" 200 49


{'code': 0, 'msg': '', 'data': {'job_id': 10046}}

In [3]:
# 查询抽样任务
sampling_job_details = submitter.get_sampling_job_details(job_id)
sampled_path = sampling_job_details.sampled_path
sampling_job_details.to_dict()
sampled_path

request: http://localhost:8000/v1/sampling/query/job/
2021-03-23 16:26:58,777 DEBUG connectionpool.py[231] Starting new HTTP connection (1): localhost:8000
2021-03-23 16:26:58,782 DEBUG connectionpool.py[461] http://localhost:8000 "POST /v1/sampling/query/job/ HTTP/1.1" 200 462


'hdfs://localhost:9000/dataset/titanic/train.csv-sampled-1616488010.6182182'

In [4]:
# 简单的读取抽样之后的文件
from pyspark.sql import SparkSession
from sparksampling.config import SPARK_CONF

conf = SPARK_CONF
spark = SparkSession.builder.config(conf=conf).getOrCreate()
df = spark.read.csv(sampled_path, header=True)
df.toPandas()
# 可以在这后面做数据分析，或试试看下面的统计、评估功能

# data analyse here


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,None,S
1,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,None,S
2,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,None,S
3,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.1333,None,S
4,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,30.0708,None,C
...,...,...,...,...,...,...,...,...,...,...,...,...
789,883,0,3,"Dahlberg, Miss. Gerda Ulrika",female,22,0,0,7552,10.5167,None,S
790,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39,0,5,382652,29.125,None,Q
791,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39,0,5,382652,29.125,None,Q
792,889,0,3,"""Johnston, Miss. Catherine Helen """"Carrie""""""",female,None,1,2,W./C. 6607,23.45,None,S


In [54]:
# 统计原数据集
submitter.get_statistics(path=dataset_uri, file_type=FILE_TYPE_CSV,with_header=True).to_pandas()

request: http://localhost:8000/v1/evaluation/statistics/
2021-03-23 14:59:29,518 DEBUG connectionpool.py[231] Starting new HTTP connection (1): localhost:8000
2021-03-23 14:59:29,790 DEBUG connectionpool.py[461] http://localhost:8000 "POST /v1/evaluation/statistics/ HTTP/1.1" 200 1414


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
summary,,,,,,,,,,,,
count,891,891,891,891,891,714,891,891,891,891,204,889
mean,446.0,0.3838383838383838,2.308641975308642,None,None,29.69911764705882,0.5230078563411896,0.38159371492704824,260318.54916792738,32.2042079685746,None,None
stddev,257.3538420152301,0.48659245426485753,0.8360712409770491,None,None,14.526497332334035,1.1027434322934315,0.8060572211299488,471609.26868834975,49.69342859718089,None,None
min,1,0,1,"""Andersson, Mr. August Edvard (""""Wennerstrom"""")""",female,0.42,0,0,110152,0,A10,C
max,99,1,3,"van Melkebeke, Mr. Philemon",male,9,8,6,WE/P 5735,93.5,T,S


In [55]:
# 统计抽样后的数据集
data = submitter.get_statistics(job_id=job_id, from_sampling=True, file_type=FILE_TYPE_CSV,with_header=True).to_pandas()
data

request: http://localhost:8000/v1/evaluation/statistics/
2021-03-23 14:59:31,341 DEBUG connectionpool.py[231] Starting new HTTP connection (1): localhost:8000
2021-03-23 14:59:31,674 DEBUG connectionpool.py[461] http://localhost:8000 "POST /v1/evaluation/statistics/ HTTP/1.1" 200 1430


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
summary,,,,,,,,,,,,
count,766,766,766,766,766,629,766,766,766,766,182,763
mean,455.29634464751956,0.3838120104438642,2.3237597911227152,None,None,29.986104928457866,0.47911227154047,0.34856396866840733,257857.6390328152,28.46103054830285,None,None
stddev,260.4302315959202,0.48663076543580014,0.8352586165992867,None,None,14.300856437154597,1.0510908098954141,0.7566121776748826,471471.50937359943,39.20021140870341,None,None
min,1,0,1,"""Andersson, Mr. August Edvard (""""Wennerstrom"""")""",female,0.42,0,0,110152,0,A10,C
max,99,1,3,"van Melkebeke, Mr. Philemon",male,9,8,6,WE/P 5735,91.0792,T,S


In [56]:
# 提交评估任务
evaluation_job = submitter.submit_evaluation_job(compare_job_id=job_id, file_type=FILE_TYPE_CSV)
evaluation_job.to_dict()

request: http://localhost:8000/v1/evaluation/job/
2021-03-23 14:59:34,614 DEBUG connectionpool.py[231] Starting new HTTP connection (1): localhost:8000
2021-03-23 14:59:34,632 DEBUG connectionpool.py[461] http://localhost:8000 "POST /v1/evaluation/job/ HTTP/1.1" 200 49


{'code': 0, 'msg': '', 'data': {'job_id': 50019}}

In [57]:
evaluation_job_id = evaluation_job.job_id
evaluation_job_id

50019

In [58]:
# 查看各属性评估得分
evaluation_job_data = submitter.get_evaluation_job_details(job_id=evaluation_job_id)
evaluation_job_data.to_pandas()

request: http://localhost:8000/v1/evaluation/query/job/
2021-03-23 14:59:43,628 DEBUG connectionpool.py[231] Starting new HTTP connection (1): localhost:8000
2021-03-23 14:59:43,634 DEBUG connectionpool.py[461] http://localhost:8000 "POST /v1/evaluation/query/job/ HTTP/1.1" 200 2623


,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
count,629,182,763,766,766,766,766,766,766,766,766,766
mean,29.986104928457866,None,None,28.46103054830285,None,0.34856396866840733,455.29634464751956,2.3237597911227152,None,0.47911227154047,0.3838120104438642,257857.6390328152
stddev,14.300856437154597,None,None,39.20021140870341,None,0.7566121776748826,260.4302315959202,0.8352586165992867,None,1.0510908098954141,0.48663076543580014,471471.50937359943
min,0.42,A10,C,0,"""Andersson, Mr. August Edvard (""""Wennerstrom"""")""",0,1,1,female,0,0,110152
max,9,T,S,91.0792,"van Melkebeke, Mr. Philemon",6,99,3,male,8,1,WE/P 5735
mean_bias,0.00966316,-1,-1,0.116233,-1,0.0865574,0.0208438,0.00654836,-1,0.0839291,6.87096e-05,0.00945346
stddev_bias,0.0155331,-1,-1,0.211159,-1,0.0613419,0.0119539,0.000971956,-1,0.0468401,7.87336e-05,0.000292105
score,98.7402,-1,-1,83.6304,-1,92.605,98.3601,99.624,-1,93.4615,99.9926,99.5127


In [59]:
df = evaluation_job_data.to_pandas()
score_list = df.loc['score'].to_list()
while -1 in score_list:
    score_list.remove(-1)
score_list
import numpy as np
np.mean(score_list)

95.74082919046297